# Verificación Calidad de los datos


En este notebook, se va a verificar la calidad de los datos y se va a generar un dataset.
La verificación de calidad va a consistir en:
* Completitud
    * Evaluación de valores nulos.
    * Evaluación de completitud serie temporal.
    * Evaluación de rango de fecha
* Consistencia
    * Filas duplicadas
    * Claves únicas
    * Integridad referencial
* Exactitud
    * Evaluación de formato válido
    * Valores outliers


Establecemos los umbrales de aceptación:

In [1]:
completitud_n = 0.2 #menor
completitud_st = 0.1 #menor
completitud_f = 5 #mayor
filas_duplicas = 0 #
valores_outliers = 0.0 
formato_invalido = 0 #variables
claves_unicas = 0.0
integridad = 0.0

## Importación de librerías

In [2]:
import pandas as pd

## Lectura de datasets

In [3]:
df_hist = pd.read_csv("../../data/raw/historico_completo.csv")

df_hist.head()

,fecha,dia,tempMedia,tempMax,horMinTempMax,tempMin,horMinTempMin,humedadMedia,humedadMax,horMinHumMax,...,velVientoMax,horMinVelMax,dirVientoVelMax,radiacion,precipitacion,bateria,fechaUtlMod,et0,provincia_id,codigoEstacion
0,2005-01-01,1,9.07,18.58,14:30,0.658,07:20,67.31,87.2,07:40,...,5.008,11:51,338.1,11.110,0.0,12.93,2005-01-02T07:34:55.000+0100,1.495588,11,1
1,2005-01-02,2,9.02,12.17,14:50,5.810,21:30,84.70,91.8,23:30,...,4.106,15:44,333.5,4.173,0.0,12.92,2005-01-03T07:37:28.000+0100,0.776324,11,1
2,2005-01-03,3,8.18,15.91,14:20,3.196,06:10,80.60,92.9,11:50,...,5.174,14:43,318.2,9.090,0.2,12.94,2005-01-04T07:34:53.000+0100,1.327342,11,1
3,2005-01-04,4,10.55,18.26,12:40,3.608,07:30,68.63,87.2,06:50,...,7.680,12:48,116.6,10.400,0.0,12.93,2005-01-05T07:34:53.000+0100,1.796508,11,1
4,2005-01-05,5,9.82,17.78,14:50,2.802,07:30,67.62,87.5,06:10,...,4.508,11:12,145.7,11.090,0.0,12.93,2005-01-06T07:35:04.000+0100,1.371563,11,1


In [4]:
df_est = pd.read_csv("../../data/raw/estaciones.csv")
df_est.head()

,codigoEstacion,nombre,bajoplastico,activa,visible,longitud,latitud,altitud,xutm,yutm,huso,provincia_id,provincia_nombre
0,10,Adra,False,True,True,025932000W,364448000N,2,500683.0,4066780.0,30,4,Almería
1,2,Almería,False,True,True,022408000W,365007000N,5,553282.0,4076780.0,30,4,Almería
2,8,Cuevas de Almanzora,False,True,True,014801000W,371524000N,28,606367.0,4124030.0,30,4,Almería
3,5,Fiñana,False,True,True,025019000W,370924000N,958,514311.0,4112270.0,30,4,Almería
4,7,Huércal-Overa,False,True,True,015303000W,372444000N,303,598735.0,4141210.0,30,4,Almería


## Dimensión de completitud  



**ANÁLISIS DE VALORES NULOS**

In [53]:
nulos_x_columna_est = df_est.isna().sum()


print(f"Cantidad de filas que tienen valores nulos por atributo en los metadatos de las estaciones:\n{nulos_x_columna_est}\n")

Cantidad de filas que tienen valores nulos por atributo en los metadatos de las estaciones:
codigoEstacion      0
nombre              0
bajoplastico        0
activa              0
visible             0
longitud            0
latitud             0
altitud             0
xutm                0
yutm                0
huso                0
provincia_id        0
provincia_nombre    0
dtype: int64



In [54]:
cantidad_filas_est = df_est.shape[0]
cantidad_filas_hist = df_hist.shape[0]
cantidad_columnas = len(df_est.axes[1])
df_est['completitud_fila'] = (df_est.isnull().sum(axis=1) / cantidad_columnas)
problemas = df_est[df_est['completitud_fila'] > completitud_n]
completitud_f = problemas.shape[0]
print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f] - estaciones - :")
print(f"{completitud_n} ({round((completitud_n  / cantidad_filas_est) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - estaciones - :
0.2 (0.2)%


En el caso del histórico, no tiene sentido analizarlo a nivel global. Tenemos que analizar los valores faltantes por estación, para identificar estaciones problemáticas.

In [56]:
total_nulos_por_estacion = df_hist.isnull().groupby([df_hist['codigoEstacion'], df_hist['provincia_id']]).sum().sum(axis=1)
total_celdas_por_estacion = df_hist.groupby(['codigoEstacion', 'provincia_id']).size() * len(df_hist.columns)
porcentaje_nulos_total = (total_nulos_por_estacion / total_celdas_por_estacion)
resultado = porcentaje_nulos_total.round(2).sort_values(ascending=False)
print(f"Porcentaje de filas que tienen valores nulos por atributo en el histórico por estación:\n{resultado}\n")

Porcentaje de filas que tienen valores nulos por atributo en el histórico por estación:
codigoEstacion  provincia_id
102             21              0.25
103             21              0.25
12              18              0.01
17              23              0.01
1               23              0.00
                                ... 
101             41              0.00
102             18              0.00
                23              0.00
103             23              0.00
104             23              0.00
Length: 100, dtype: float64



In [57]:
resultado>completitud_n

codigoEstacion  provincia_id
102             21               True
103             21               True
12              18              False
17              23              False
1               23              False
                                ...  
101             41              False
102             18              False
                23              False
103             23              False
104             23              False
Length: 100, dtype: bool

In [58]:
print(f"Solo 2 estaciones ({2/len(resultado)*100}% de las estaciones) tienen más de un 20% de valores nulos en sus filas")

Solo 2 estaciones (2.0% de las estaciones) tienen más de un 20% de valores nulos en sus filas


**ANÁLISIS SERIE TEMPORAL**

In [59]:
def verificacion_serie_temporal(station_data):
       
    fecha_inicio = station_data.index.min()
    fecha_fin = station_data.index.max()
    
    rango_completo = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
    
    dias_esperados = len(rango_completo)
    dias_reales = len(station_data)
    
    porcentaje_faltante = (1 - (dias_reales / dias_esperados)) * 100
    
    return pd.Series({
        'fecha_inicio': fecha_inicio,
        'fecha_fin': fecha_fin,
        'dias_esperados': dias_esperados,
        'dias_reales': dias_reales,
        '': dias_reales/365,
        'porcentaje_faltante': porcentaje_faltante
    })

df_index = df_hist.set_index('fecha')
df_verificado = df_index.groupby(['codigoEstacion','provincia_id']).apply(verificacion_serie_temporal)


df_verificado.sort_values('porcentaje_faltante', ascending=False).round(2)

/tmp/ipykernel_544263/667874047.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_verificado = df_index.groupby(['codigoEstacion','provincia_id']).apply(verificacion_serie_temporal)


,,fecha_inicio,fecha_fin,dias_esperados,dias_reales,,porcentaje_faltante
codigoEstacion,provincia_id,,,,,,
102,21,2008-11-20,2025-08-01,6099,4362,11.95,28.48
103,21,2011-12-01,2025-08-01,4993,3836,10.51,23.17
8,4,2005-01-01,2025-08-01,7518,6812,18.66,9.39
5,11,2005-01-01,2025-08-01,7518,7028,19.25,6.52
101,23,2006-04-06,2025-08-01,7058,6756,18.51,4.28
...,...,...,...,...,...,...,...
1,18,2005-01-01,2025-08-01,7518,7514,20.59,0.05
15,41,2005-01-01,2025-08-01,7518,7514,20.59,0.05
8,21,2005-01-01,2025-08-01,7518,7515,20.59,0.04


In [ ]:
num = df_verificado[(df_verificado["porcentaje_faltante"]>10)==True].shape[0]

print(f"{num} estaciones tienen más del 10% de registros ausentes en el rango temporal indicado.")

2 estaciones tienes más del 10% de registros ausentes en el rango temporal indicado.


**RANGO DE FECHA**

In [ ]:
num_reg_estacion = df_hist.groupby(['provincia_id','codigoEstacion']).size().sort_values(ascending=False)
num_reg_estacion = num_reg_estacion.to_frame().reset_index()
num_reg_estacion.columns = ['provincia', 'codigo', 'registros']
num_reg_estacion['años'] = num_reg_estacion['registros']/365
num_reg_estacion[num_reg_estacion['años']<5]

,provincia,codigo,registros,años
99,21,12,303,0.830137


In [80]:
menor5 = num_reg_estacion[num_reg_estacion['años']<5].shape[0]
N = num_reg_estacion.shape[0]
print(f"{num_reg_estacion[num_reg_estacion['años']<5].shape[0]} estación tiene menos de 5 años de registros")
print(f'El {menor5/N*100}% de las estaciones tiene menos de 5 años de registros')

1 estación tiene menos de 5 años de registros
El 1.0% de las estaciones tiene menos de 5 años de registros


## Dimensión de consistencia

**DUPLICADOS**

In [82]:
print(f"Hay {df_est.duplicated().sum()} instancias duplicadas en los metadatos de la estación. Supone un {round(df_est.duplicated().sum()/df_est.shape[0]*100,2)}%")
print(f"Hay {df_hist.duplicated().sum()} instancias duplicadas en el histórico. Supone un {round(df_hist.duplicated().sum()/df_hist.shape[0]*100,2)}%")

Hay 0 instancias duplicadas en los metadatos de la estación. Supone un 0.0%
Hay 107 instancias duplicadas en el histórico. Supone un 0.02%


**CLAVES ÚNICAS**

In [85]:
total_filas = len(df_est)
filas_duplicadas = df_est.duplicated(subset=['codigoEstacion', 'provincia_id']).sum()
claves_unicas_pct = (filas_duplicadas / total_filas) * 100 if total_filas > 0 else 0

print(f"Filas con clave ('codigoEstacion' y 'provincia_id') duplicada: {filas_duplicadas}")
print(f"Porcentaje (claves_unicas): {claves_unicas_pct:.2f}%")

Filas con clave ('codigoEstacion' y 'provincia_id') duplicada: 0
Porcentaje (claves_unicas): 0.00%


In [84]:
total_filas = len(df_hist)
filas_duplicadas = df_hist.duplicated(subset=['codigoEstacion', 'provincia_id' ,'fecha']).sum()
claves_unicas_pct = (filas_duplicadas / total_filas) * 100 if total_filas > 0 else 0

print(f"Filas con clave ('codigoEstacion', 'provincia_id'  y 'fecha') duplicada: {filas_duplicadas}")
print(f"Porcentaje (claves_unicas): {claves_unicas_pct:.2f}%")

Filas con clave ('codigoEstacion', 'provincia_id'  y 'fecha') duplicada: 107
Porcentaje (claves_unicas): 0.02%


Se corresponde con las filas detectadas como réplicas anteriormente. No hay nuevos registros en los que no se cumpla la unicidad de la clave.

**INTEGRIDAD REFERENCIAL**

In [91]:
df_integrado = pd.merge(df_hist, df_est, on=["codigoEstacion", "provincia_id"],how='left', indicator=True)
mediciones_sin_estacion = df_integrado[df_integrado['_merge'] == 'left_only'].shape[0]
mediciones_sin_estacion
print(f"Filas sin una correspondencia exacta de (codigoEstacion, provincia_id) en el fichero de metadatos: {mediciones_sin_estacion}")
print(f"Porcentaje de errores de integridad referencial: {round(mediciones_sin_estacion/len(df_hist)*100,2)}%")

Filas sin una correspondencia exacta de (codigoEstacion, provincia_id) en el fichero de metadatos: 0
Porcentaje de errores de integridad referencial: 0.0%


## Dimensión de exactitud

**FORMATO**

In [96]:
print(f"Formato esperado para fecha: Date. Formato obtenido: { df_hist['fecha'].dtype}")
if df_hist['fecha'].dtype == "datetime64":
    print("Todas las variables tienen el formato esperado.")
else:
    print("1 variable no tiene el formato esperado.")

Formato esperado para fecha: Date. Formato obtenido: object
1 variable no tiene el formato esperado.


**VALORES OUTLIER**

In [8]:
var_numericas= df_hist.select_dtypes('float').columns
def deteccion_outlier(data, var):

    Q1 = data[var].quantile(0.25)
    Q3 = data[var].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = (data[var] < lower) | (data[var] > upper)
    return outliers

outliers_por_estacion=df_hist.groupby(['provincia_id', 'codigoEstacion'])[var_numericas].apply(
    lambda x: deteccion_outlier(x, var_numericas).sum()
)
round(outliers_por_estacion.sum(axis=0).sort_values(ascending=False)/len(df_hist),2)*100

precipitacion      18.0
velViento           5.0
bateria             3.0
velVientoMax        3.0
humedadMax          2.0
dirVientoVelMax     2.0
dirViento           1.0
humedadMin          1.0
humedadMedia        0.0
et0                 0.0
tempMin             0.0
tempMax             0.0
tempMedia           0.0
radiacion           0.0
dtype: float64